In [389]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [390]:
data=pd.read_csv('상품명 색깔 카테고리 RGB .csv')
data.head()

,상품명,색깔,카테고리,RGB,Unnamed: 4,Unnamed: 5
0,젤르 린넨 라운드가디건,화이트,가디건,"RGB(255, 255, 255)",NaN,NaN
1,아이디 슬림핏 골지가디건,화이트,가디건,"RGB(255, 255, 255)",NaN,NaN
2,사샤 앙고라 가디건,화이트,가디건,"RGB(255, 255, 255)",NaN,NaN
3,플로이 나시&가디건set,화이트,가디건,"RGB(255, 255, 255)",NaN,NaN
4,제니스 배색가디건,화이트,가디건,"RGB(255, 255, 255)",NaN,NaN


In [391]:
data=data[['상품명','색깔','카테고리','RGB']]
data.head()

,상품명,색깔,카테고리,RGB
0,젤르 린넨 라운드가디건,화이트,가디건,"RGB(255, 255, 255)"
1,아이디 슬림핏 골지가디건,화이트,가디건,"RGB(255, 255, 255)"
2,사샤 앙고라 가디건,화이트,가디건,"RGB(255, 255, 255)"
3,플로이 나시&가디건set,화이트,가디건,"RGB(255, 255, 255)"
4,제니스 배색가디건,화이트,가디건,"RGB(255, 255, 255)"


In [392]:
data.head(2)

,상품명,색깔,카테고리,RGB
0,젤르 린넨 라운드가디건,화이트,가디건,"RGB(255, 255, 255)"
1,아이디 슬림핏 골지가디건,화이트,가디건,"RGB(255, 255, 255)"


In [393]:
data.RGB.head

<bound method NDFrame.head of 0       RGB(255, 255, 255)
1       RGB(255, 255, 255)
2       RGB(255, 255, 255)
3       RGB(255, 255, 255)
4       RGB(255, 255, 255)
               ...        
3531        RGB(133,94,35)
3532            RGB(0,0,0)
3533       RGB(133,104,62)
3534      RGB(226,199,170)
3535            RGB(0,0,0)
Name: RGB, Length: 3536, dtype: object>

In [394]:
data["RGB"] = [str(x).replace('RGB','')for x in data["RGB"]]
data.head()

,상품명,색깔,카테고리,RGB
0,젤르 린넨 라운드가디건,화이트,가디건,"(255, 255, 255)"
1,아이디 슬림핏 골지가디건,화이트,가디건,"(255, 255, 255)"
2,사샤 앙고라 가디건,화이트,가디건,"(255, 255, 255)"
3,플로이 나시&가디건set,화이트,가디건,"(255, 255, 255)"
4,제니스 배색가디건,화이트,가디건,"(255, 255, 255)"


In [395]:
data["RGB"] = [str(x).replace('(','').replace(')','') for x in data["RGB"]]
data.head()

,상품명,색깔,카테고리,RGB
0,젤르 린넨 라운드가디건,화이트,가디건,"255, 255, 255"
1,아이디 슬림핏 골지가디건,화이트,가디건,"255, 255, 255"
2,사샤 앙고라 가디건,화이트,가디건,"255, 255, 255"
3,플로이 나시&가디건set,화이트,가디건,"255, 255, 255"
4,제니스 배색가디건,화이트,가디건,"255, 255, 255"


In [396]:
count_vector = CountVectorizer(ngram_range=(1,1))

In [397]:
c_vector_RGB=count_vector.fit_transform(data['RGB'])
c_vector_CATE=count_vector.fit_transform(data['카테고리'])

In [398]:
c_vector_RGB.shape
c_vector_CATE.shape

(3536, 55)

In [399]:
genre_c_simR=cosine_similarity(c_vector_RGB,c_vector_RGB).argsort()[:,::-1]

In [400]:
genre_c_simC=cosine_similarity(c_vector_CATE,c_vector_CATE).argsort()[:,::-1]

In [401]:
genre_c_simC.shape

(3536, 3536)

In [402]:
def get_recommend_item_list(df,상품명,top=30):
    #특정 영화와 비슷한 영화를추천해야 하기에 '특정 영화' 정보를 추출
    target_item_index=df[df['상품명']==상품명].index.values
    
    #코사인 유사도 중 비슷한 코사인 유사도를 가진 정보 추출
    sim_indexR=genre_c_simR[target_item_index,:top].reshape(-1)
    sim_indexC=genre_c_simC[target_item_index,:top].reshape(-1)
    
    #본인 제외
    sim_indexR=sim_indexR[sim_indexR != target_item_index]
    sim_indexC=sim_indexC[sim_indexC != target_item_index]
    
    #data frame로 만들고 vote_count로 정렬한뒤 return
    resultR=df.iloc[sim_indexR].sort_values('RGB',ascending=False)[:10]
    resultC=df.iloc[sim_indexC].sort_values('RGB',ascending=False)[:10]
    return resultR

In [404]:
get_recommend_item_list(data,'젤르 린넨 라운드가디건')

<ipython-input-402-4ccebcfad4ad>:10: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  sim_indexR=sim_indexR[sim_indexR != target_item_index]
<ipython-input-402-4ccebcfad4ad>:11: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  sim_indexC=sim_indexC[sim_indexC != target_item_index]


ValueError: Buffer has wrong number of dimensions (expected 1, got 2)

In [405]:
target_item_index=data[data['상품명']=='아이디 슬림핏 골지가디건'].index.values

In [406]:
target_item_index

array([  1, 235], dtype=int64)

In [407]:
np.reshape(target_item_index, (-1))

array([  1, 235], dtype=int64)

In [408]:
sim_indexR=genre_c_simR[target_item_index,:10].reshape(-1)
sim_indexC=genre_c_simC[target_item_index,:10].reshape(-1)

In [409]:
sim_indexR

array([   0, 3414, 2408, 2407, 1748, 2406, 2405,  692, 2664,  684,  236,
        430, 3223,  235,  234,  233,  232,  710, 1441, 1421], dtype=int64)

In [410]:
sim_indexC

array([  0, 183, 189, 188, 187, 186, 185, 184, 182, 191,   0, 183, 189,
       188, 187, 186, 185, 184, 182, 191], dtype=int64)

In [411]:
sim_indexR=sim_indexR[sim_indexR != target_item_index]
sim_indexC=sim_indexC[sim_indexC != target_item_index]

<ipython-input-411-112ae1a9438c>:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  sim_indexR=sim_indexR[sim_indexR != target_item_index]
<ipython-input-411-112ae1a9438c>:2: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  sim_indexC=sim_indexC[sim_indexC != target_item_index]


In [412]:
sim_indexR, sim_indexC

(array([[   0, 3414, 2408, 2407, 1748, 2406, 2405,  692, 2664,  684,  236,
          430, 3223,  235,  234,  233,  232,  710, 1441, 1421]],
       dtype=int64),
 array([[  0, 183, 189, 188, 187, 186, 185, 184, 182, 191,   0, 183, 189,
         188, 187, 186, 185, 184, 182, 191]], dtype=int64))

In [413]:
resultR=data.iloc[sim_indexR].sort_values('RGB',ascending=False)[:10]
resultC=data.iloc[sim_indexC].sort_values('RGB',ascending=False)[:10]

ValueError: Buffer has wrong number of dimensions (expected 1, got 2)

In [414]:
resultR

,상품이름,카테고리,RGB
0,젤르 린넨 라운드가디건-화이트,가디건,"255, 255, 255"
2408,오멜리 꼬임 스트랩샌들-화이트,샌들,"255, 255, 255"
2407,러프 스트랩샌들-화이트,샌들,"255, 255, 255"
1748,코노 셔츠원피스(벨트set)-화이트,미니원피스,"255, 255, 255"
2406,팬시 오픈토샌들-화이트,샌들,"255, 255, 255"
2405,오멜리 꼬임 스트랩샌들-화이트,샌들,"255, 255, 255"
692,메쉬 베이직셔츠-화이트,니트/스웨터,"255, 255, 255"
2664,에즈 블로퍼-화이트,슬리퍼/쪼리,"255, 255, 255"
684,웰던 라운드니트-화이트,니트/스웨터,"255, 255, 255"


array 행, 열 모두 1인경우에만 값이 나옴

In [415]:
data.head()

,상품명,색깔,카테고리,RGB
0,젤르 린넨 라운드가디건,화이트,가디건,"255, 255, 255"
1,아이디 슬림핏 골지가디건,화이트,가디건,"255, 255, 255"
2,사샤 앙고라 가디건,화이트,가디건,"255, 255, 255"
3,플로이 나시&가디건set,화이트,가디건,"255, 255, 255"
4,제니스 배색가디건,화이트,가디건,"255, 255, 255"


In [416]:
data['상품이름']=data['상품명']+'-'+data['색깔']

In [417]:
data.head()

,상품명,색깔,카테고리,RGB,상품이름
0,젤르 린넨 라운드가디건,화이트,가디건,"255, 255, 255",젤르 린넨 라운드가디건-화이트
1,아이디 슬림핏 골지가디건,화이트,가디건,"255, 255, 255",아이디 슬림핏 골지가디건-화이트
2,사샤 앙고라 가디건,화이트,가디건,"255, 255, 255",사샤 앙고라 가디건-화이트
3,플로이 나시&가디건set,화이트,가디건,"255, 255, 255",플로이 나시&가디건set-화이트
4,제니스 배색가디건,화이트,가디건,"255, 255, 255",제니스 배색가디건-화이트


In [418]:
data=data[['상품이름','카테고리','RGB']]

In [419]:
data.head()

,상품이름,카테고리,RGB
0,젤르 린넨 라운드가디건-화이트,가디건,"255, 255, 255"
1,아이디 슬림핏 골지가디건-화이트,가디건,"255, 255, 255"
2,사샤 앙고라 가디건-화이트,가디건,"255, 255, 255"
3,플로이 나시&가디건set-화이트,가디건,"255, 255, 255"
4,제니스 배색가디건-화이트,가디건,"255, 255, 255"


In [420]:
count_vector = CountVectorizer(ngram_range=(1,1))

In [421]:
c_vector_RGB=count_vector.fit_transform(data['RGB'])
c_vector_CATE=count_vector.fit_transform(data['카테고리'])

In [422]:
genre_c_simR=cosine_similarity(c_vector_RGB,c_vector_RGB).argsort()[:,::-1]
genre_c_simC=cosine_similarity(c_vector_CATE,c_vector_CATE).argsort()[:,::-1]

In [423]:
target_item_index=data[data['상품이름']=='워너블 베이직셔츠-화이트'].index.values
sim_indexR=genre_c_simR[target_item_index,:10].reshape(-1)
sim_indexC=genre_c_simC[target_item_index,:10].reshape(-1)

In [424]:
sim_indexR
sim_indexC

array([3132, 3204, 3191, 3192, 3193, 3194, 3195, 3196, 3197, 3198],
      dtype=int64)

In [425]:
sim_indexR=sim_indexR[sim_indexR != target_item_index]
sim_indexC=sim_indexC[sim_indexC != target_item_index]

In [426]:
sim_indexR, sim_indexC

(array([   0, 2408, 2407, 1748, 2406, 2405,  692, 2664,  684], dtype=int64),
 array([3132, 3204, 3191, 3192, 3193, 3194, 3195, 3196, 3197, 3198],
       dtype=int64))

In [427]:
resultR=data.iloc[sim_indexR].sort_values('RGB',ascending=False)[:10]
resultC=data.iloc[sim_indexC].sort_values('RGB',ascending=False)[:10]


In [428]:
resultR

,상품이름,카테고리,RGB
0,젤르 린넨 라운드가디건-화이트,가디건,"255, 255, 255"
2408,오멜리 꼬임 스트랩샌들-화이트,샌들,"255, 255, 255"
2407,러프 스트랩샌들-화이트,샌들,"255, 255, 255"
1748,코노 셔츠원피스(벨트set)-화이트,미니원피스,"255, 255, 255"
2406,팬시 오픈토샌들-화이트,샌들,"255, 255, 255"
2405,오멜리 꼬임 스트랩샌들-화이트,샌들,"255, 255, 255"
692,메쉬 베이직셔츠-화이트,니트/스웨터,"255, 255, 255"
2664,에즈 블로퍼-화이트,슬리퍼/쪼리,"255, 255, 255"
684,웰던 라운드니트-화이트,니트/스웨터,"255, 255, 255"


In [429]:
def get_recommend_item_list(df,상품이름,top=10):
    #특정 영화와 비슷한 영화를추천해야 하기에 '특정 영화' 정보를 추출
    target_item_index=df[df['상품이름']==상품이름].index.values
    
    #코사인 유사도 중 비슷한 코사인 유사도를 가진 정보 추출
    sim_indexR=genre_c_simR[target_item_index,:top].reshape(-1)
    sim_indexC=genre_c_simC[target_item_index,:top].reshape(-1)

    #본인 제외
    sim_indexR=sim_indexR[sim_indexR != target_item_index]
    sim_indexC=sim_indexC[sim_indexC != target_item_index]
    
    #data frame로 만들고 vote_count로 정렬한뒤 return
    resultR=df.iloc[sim_indexR].sort_values('RGB',ascending=False)[:10]
    resultC=df.iloc[sim_indexC].sort_values('RGB',ascending=False)[:10]
    return resultR

In [431]:
data.iloc[10:30]

,상품이름,카테고리,RGB
10,롤리 꽈배기 가디건-핑크,가디건,"255, 191, 202"
11,클리어 여름가디건-핑크,가디건,"255, 191, 202"
12,젤르 린넨 라운드가디건-핑크,가디건,"255, 191, 202"
13,몽트 꽈배기가디건-핑크,가디건,"255, 191, 202"
14,노이드 썸머 부클가디건-핑크,가디건,"255, 191, 202"
15,에트나 알파카 크롭가디건-핑크,가디건,"255, 191, 202"
16,히얼 진주 골지반팔가디건-핑크,가디건,"255, 191, 202"
17,클로니 나시&가디건set-핑크,가디건,"255, 191, 202"
18,플로이 나시&가디건set-핑크,가디건,"255, 191, 202"
19,멜츠 배색 크롭가디건-핑크,가디건,"255, 191, 202"


In [433]:
get_recommend_item_list(data,'플로이 나시&가디건set-핑크')

,상품이름,카테고리,RGB
2156,루세 시스루 셔링블라우스-핑크,블라우스,"255, 191, 202"
2166,코렌 프릴 랩블라우스-핑크,블라우스,"255, 191, 202"
2171,나이브 프릴블라우스-핑크,블라우스,"255, 191, 202"
1669,네스 꽈배기 랩원피스-핑크,미니원피스,"255, 191, 202"
2170,엘리스 새틴 셔링블라우스-핑크,블라우스,"255, 191, 202"
2169,구르미 퍼프 셔링블라우스-핑크,블라우스,"255, 191, 202"
2168,아를 셔링 프릴블라우스-핑크,블라우스,"255, 191, 202"
3157,프림 스퀘어 골지티-핑크,티셔츠,"255, 191, 202"
2167,[기획특가]데이즈 오프숄더블라우스-핑크,블라우스,"255, 191, 202"
3395,[La belle rose]파우더 슬림핏 랩 티셔츠-핑크,티셔츠,"255, 191, 202"
